In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def generate_poisson_arrivals(T, frequency):

    arrivals = []
    current_time = 0

    # Generate arrivals until we exceed T
    while True:
        interarrival = np.random.exponential(1/frequency)
        current_time += interarrival

        if current_time < T:
            arrivals.append(current_time)
        else:
            break

    return np.array(arrivals)

In [3]:
def generate_multifractal_measure(T, frequencies, multiplier_distribution, depth=10):

    n_points = 1000  # Resolution of the measure
    x = np.linspace(0, T, n_points + 1)
    measure = np.ones(n_points)

    for i, freq in enumerate(frequencies):
        arrivals = generate_poisson_arrivals(T, freq)

        # Add endpoints
        arrival_points = np.sort(np.concatenate([[0], arrivals, [T]]))

        for j in range(len(arrival_points) - 1):
            start_idx = np.searchsorted(x, arrival_points[j], side='left')
            end_idx = np.searchsorted(x, arrival_points[j+1], side='right')

            m = multiplier_distribution()

            measure[start_idx:end_idx] *= m

    interval_width = T / n_points
    total_mass = np.sum(measure * interval_width)
    normalized_measure = measure / total_mass

    cdf = np.zeros(n_points + 1)
    np.cumsum(normalized_measure * interval_width, out=cdf[1:])

    return x, measure, cdf



def generate_brownian_motion(T=1.0, N=1000, seed=None):
    if seed is not None:
        np.random.seed(seed)

    t = np.linspace(0, T, N)
    dt = T / (N - 1)

    dW = np.random.normal(0, np.sqrt(dt), N-1)

    B = np.zeros(N)
    B[1:] = np.cumsum(dW)

    return t, B

In [4]:
def compound_process(t, B, x_cdf, cdf):
    theta_t = np.interp(t, x_cdf, cdf)
    X = np.interp(theta_t, np.linspace(0, 1, len(B)), B)
    return X

def lognormal_multiplier_generator(mu=-0.1, sigma=0.3):

    adjusted_mu = -sigma**2/2

    def generator():
        return np.random.lognormal(adjusted_mu, sigma)

    return generator